# UDF

In [3]:
import duckdb
import concurrent.futures
import openai

# Set your OpenAI API key
openai.api_key = 'your-api-key'
ENGINE = 'your-open-ai-engine'

from typing import List

In [ ]:
def llm(query: str, context: List[str]):
    """
    query: e.x. find the artist mentioned in the {review}
    context: e.x. List[review]
    """
    
#     set_model_key()
#     llm_chain = llm_dict[query]
#     output = llm_chain.batch(context)

    def invoke_llm(query: str, context: str):
        """
        query: e.x. find the artist mentioned in the {review}
        context: e.x. review 
        """
        set_model_key()

        response_format = """
            You are a data analysis assistant who will respond with the artist name mentioned on reviews inputted. Answer with only the name.
            For instance:
            - If the review is 'I love the songs by Taylor Swift.', the answer is 'Taylor Swift'.
            - If the review is 'This album reminds me of the Beatles', the answer is 'the Beatles'.
        """


        response = openai.ChatCompletion.create(
            engine=DEPLOYMENT_NAME, # engine = "deployment_name".
            messages=[
                {"role": "system", "content": response_format},
                {"role": "user", "content": context}
            ]
        )

        output = response['choices'][0]['message']['content']
        return output

    
    responses = []
    # We can use a with statement to ensure threads are cleaned up promptly
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        # Start the load operations and mark each future with its URL
        future_to_response = {executor.submit(invoke_llm, query, row): row for row in context}
        for future in concurrent.futures.as_completed(future_to_response):
            row = future_to_response[future]
            try:
                data = future.result()
                responses.append(data)
            except Exception as exc:
                print('%r generated an exception: %s' % (row, exc))
    
    
    return responses